In [60]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint

In [61]:
house_dict = []

for page in range(19, 21):
    linked = 'https://www.zillow.com/ca/condos/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"mapBounds":{"west":-127.94190190625,"east":-110.67139409375,"south":32.12531172436746,"north":42.36335181945167},"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"mapZoom":6,"filterState":{"isMultiFamily":{"value":false},"isManufactured":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"isApartment":{"value":false},"isPreMarketForeclosure":{"value":false},"isPreMarketPreForeclosure":{"value":false},"isMakeMeMove":{"value":false},"isAuction":{"value":false},"isComingSoon":{"value":false},"isSingleFamily":{"value":false}},"isListVisible":true}'
    user_agent = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            # Identify and return house data
            listing_type = str(result.a.get('aria-label').split(",")[0])
            price = result.find('div', class_="list-card-price").text
            full_address = result.find('h3', class_="list-card-addr").text
            street = result.find('h3', class_="list-card-addr").text.split(",")[0]
            city = result.find('h3', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            try:
                zip_code = result.find('h3', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            house_type = loop_result1.select('li')[0].text.split(":")[1]
            year_build = loop_result1.select('li')[1].text.split(":")[1]
#             lot = loop_result1.select('li')[5].text.split(":")[1]
#             if (str(lot.split(" ")[1]) == "sqft"):
#                 lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""))
#             elif (str(lot.split(" ")[1]) == "acres"):
#                 lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', "")) * 43560
            try:
                state = result.find('h3', class_="list-card-addr").text.split(",")[2].split(" ")[1]
                zillow_price_per_sqft = loop_result1.select('li')[6].text.split(":")[1]
            except IndexError:
                state = 'null'
                zillow_price_per_sqft = 'null'
            zestimate = loop_soup.find('span', class_="ds-estimate-value").text

            
            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "House_type": house_type,
                "Year_build": year_build,
#                 "Lot": lot,
#                 "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "Zestimate": zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            
        except AttributeError as e:
            continue

print(house_dict)

[{'Listing_type': 'New construction', 'Price': '$938,995+', 'Full_address': 'W-2A Plan, Westbourne Park at Metro Crossing', 'Street': 'W-2A Plan', 'City': 'WestbourneParkatMetroCrossing', 'State': 'null', 'Zip_code': 'null', 'Bed': '2', 'Bath': '2', 'Square_Feet': '1,235', 'House_type': 'Condo', 'Year_build': '2019', 'Zestimate': '$911,539', 'Zillow_price/sqft': 'null', 'House_website': 'https://www.zillow.com/community/westbourne-park-at-metro-crossing/2085641043_zpid/'}, {'Listing_type': 'Condo for sale', 'Price': '$325,000', 'Full_address': '1434 Marshall Rd UNIT 8, Alpine, CA 91901', 'Street': '1434 Marshall Rd UNIT 8', 'City': 'Alpine', 'State': 'CA', 'Zip_code': '91901', 'Bed': '2', 'Bath': '2', 'Square_Feet': '1,005', 'House_type': 'Condo', 'Year_build': '1991', 'Zestimate': '$321,505', 'Zillow_price/sqft': '$323', 'House_website': 'https://www.zillow.com/homedetails/1434-Marshall-Rd-UNIT-8-Alpine-CA-91901/69223363_zpid/'}, {'Listing_type': 'Condo for sale', 'Price': '$289,950',

In [62]:
house_data = pd.DataFrame(house_dict)
house_data

,Bath,Bed,City,Full_address,House_type,House_website,Listing_type,Price,Square_Feet,State,Street,Year_build,Zestimate,Zillow_price/sqft,Zip_code
0,2,2,WestbourneParkatMetroCrossing,"W-2A Plan, Westbourne Park at Metro Crossing",Condo,https://www.zillow.com/community/westbourne-pa...,New construction,"$938,995+","1,235",null,W-2A Plan,2019,"$911,539",null,null
1,2,2,Alpine,"1434 Marshall Rd UNIT 8, Alpine, CA 91901",Condo,https://www.zillow.com/homedetails/1434-Marsha...,Condo for sale,"$325,000","1,005",CA,1434 Marshall Rd UNIT 8,1991,"$321,505",$323,91901
2,2,2,Antioch,"2283 Renwick Ln, Antioch, CA 94509",Condo,https://www.zillow.com/homedetails/2283-Renwic...,Condo for sale,"$289,950","1,008",CA,2283 Renwick Ln,1988,"$289,952",$288,94509
3,1,2,SanDiego,"4057 1st Ave UNIT 405, San Diego, CA 92103",Condo,https://www.zillow.com/homedetails/4057-1st-Av...,Condo for sale,"$500,000","1,025",CA,4057 1st Ave UNIT 405,2004,"$495,519",$488,92103
4,2,2,PalmDesert,"134 Deer Spring Way, Palm Desert, CA 92211",Condo,https://www.zillow.com/homedetails/134-Deer-Sp...,Condo for sale,"$339,000","1,858",CA,134 Deer Spring Way,1983,"$328,622",$182,92211
5,1,1,Alameda,"1170 9th St APT 15, Alameda, CA 94501",Condo,https://www.zillow.com/homedetails/1170-9th-St...,Condo for sale,"$450,000",577,CA,1170 9th St APT 15,1959,"$467,231",$780,94501
6,2,2,PalmSprings,"255 E Avenida Granada UNIT 124, Palm Springs, ...",Condo,https://www.zillow.com/homedetails/255-E-Aveni...,Condo for sale,"$279,000","1,198",CA,255 E Avenida Granada UNIT 124,1985,"$273,833",$233,92264
7,1,1,SanJose,"5943 Lake Hennessy Ct, San Jose, CA 95123",Condo,https://www.zillow.com/homedetails/5943-Lake-H...,Condo for sale,"$519,000",734,CA,5943 Lake Hennessy Ct,1995,"$519,074",$707,95123
8,3,2,Anaheim,"1512 W Nottingham Ln # 21, Anaheim, CA 92802",Condo,https://www.zillow.com/homedetails/1512-W-Nott...,Condo for sale,"$440,000","1,235",CA,1512 W Nottingham Ln # 21,1982,"$440,002",$356,92802
9,2,2,SanJose,"306 Ballymore Cir, San Jose, CA 95136",Condo,https://www.zillow.com/homedetails/306-Ballymo...,Condo for sale,"$674,999","1,168",CA,306 Ballymore Cir,1997,"$675,136",$578,95136


In [63]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Q4-2019_Condo_Data_page19-20.csv", index=False, header=True)